In [112]:
import pandas as pd
import numpy as np

In [113]:
excel = pd.ExcelFile('Folds5x2_pp.xlsx')
df = pd.DataFrame()

for sheet in excel.sheet_names:
    sheetdf = excel.parse(sheetname=sheet,skiprows=0)
    df = pd.concat([df, sheetdf])

In [114]:
#df.head()
df.head()

,AT,V,AP,RH,PE
0,14.96,41.76,1024.07,73.17,463.26
1,25.18,62.96,1020.04,59.08,444.37
2,5.11,39.40,1012.16,92.14,488.56
3,20.86,57.32,1010.24,76.64,446.48
4,10.82,37.50,1009.23,96.62,473.90


In [115]:
from sklearn import cross_validation
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures

from sklearn.metrics import mean_squared_error

x = df.iloc[:,0:4]
y = df.iloc[:,4]

np.random.seed(0)
indices = np.random.permutation(len(df))
X = x.iloc[indices]
y = y.iloc[indices]
n_fold = 10 # 交差検定の回数
k_fold = cross_validation.KFold(n=len(df),n_folds = n_fold)

dim=np.arange(1,21)
lambda_=[1e-2,1e-1,5e-1,1,10,20,30,40,50,60,70,80,90,100]

i = 0
L=len(dim)*len(lambda_)
score = np.zeros((L,3))
tmp=[]
for d in dim:
    degree_=PolynomialFeatures(degree=d, interaction_only=True)
    #print(d)
    for l in lambda_:
        lin_ = Ridge(normalize=True,alpha=l)
        for train, test in k_fold:
            x_=degree_.fit_transform(X.iloc[train])
            x_test=degree_.fit_transform(X.iloc[test])
            lin_.fit(x_,y.iloc[train])

            tmp.append(mean_squared_error(y.iloc[test],lin_.predict(x_test)))

        score[i,0] = d
        score[i,1] = l
        score[i,2] = sum(tmp) / len(tmp)
        i+=1
        tmp=[]
        
print(score[np.argmin(score[:,-1])])

[  4.00000000e+00   1.00000000e-02   1.91037132e+01]
